# Regression - clim_sens and sqrt

In this notebook we create many regression models - baselines and GPSR.

We generate results files.

This notebook requires the output from the data cleaning and data exploration notebooks.

The *main* regression notebook gives main results. This notebook tries some alternate versions: climate sensitivity is truncated and we introduce sqrt to the language.


In [4]:
import time
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import sympy as sp

from sklearn.model_selection import train_test_split, cross_val_score, cross_validate
from sklearn.linear_model import ElasticNet
from sklearn.ensemble import RandomForestRegressor

from pysr import PySRRegressor # a SOTA GP SR library


In [5]:
import matplotlib
matplotlib.rcParams['pdf.fonttype'] = 42 # helps produce ACM-compliant figures
matplotlib.rcParams['ps.fonttype'] = 42

In [6]:
Xy = pd.read_csv("../outputs/data_Xy_clim_sens_filter.csv", index_col=0)

In [7]:
Xy.columns

Index(['Scenario', 'SOW', 'Temp_Limit', 'Delay', 'GDP', 'Pop',
       'Other_ESD_Drivers', 'SDR', 'Elast_ESD_Driver', 'Elast_ESD_Price',
       'CO2_Storage_Poten', 'Wind_Poten', 'Solar_Poten', 'Biomass_Poten',
       'Oil_Gas_Poten', 'Solar_PV_Inv_Cost', 'Wind_Inv_Cost',
       'Bioenergy_CCS_Inv_Cost', 'Other_Tech_Cost', 'Forcing', 'Land_Sinks',
       'Clim_Sens', 'Year', 'Temp_Change', 'Rad_Forcing', 'CO2_Conc',
       'CH4_Conc', 'N20_Conc', 'Carbon', 'CO2eq', 'Marg_CO2_Cost', 'GSupply',
       'GSupply_Bioenergy', 'GSupply_Fossil', 'GSupply_Geothermal',
       'GSupply_Nuclear', 'GSupply_Solar', 'GSupply_Tidal_Waves',
       'GSupply_Wind', 'GConsumption', 'GConsumption_Fossil',
       'GConsumption_Nuclear', 'GConsumption_Renewable', 'GCost'],
      dtype='object')

## Experimental Design

We try several target variables (ie dependent variable), several subsets of independent variables, several regression models. 

## Cross-validation

In preliminary experiments we see large differences in performance according to train-test split - therefore, we use a 5-fold cross-validation.

## Reporting

For each of those 5 folds, we report the train $R^2$ and test $R^2$.

In [8]:
# We have two sets of variables, 'small' and 'all'
# For each, we have a variant which includes the scenario-encoding variables (_inc_enc)
# and a variant which excludes them.
# The idea is to compare a small versus a large subset for each scenario.
# And then as an alternative, put all the scenarios together, with the scenario-encoding
# variables included, and compare small versus large.
xvar_subsets = {
    'small': ['SDR', 'Clim_Sens', 'Pop', 'GDP'],
    'small_inc_enc': ['Temp_Limit', 'Delay', 'SDR', 'Clim_Sens', 'Pop', 'GDP'],
    'all': ['GDP', 'Pop',
       'Other_ESD_Drivers', 'SDR', 'Elast_ESD_Driver', 'Elast_ESD_Price',
       'CO2_Storage_Poten', 'Wind_Poten', 'Solar_Poten', 'Biomass_Poten',
       'Oil_Gas_Poten', 'Solar_PV_Inv_Cost', 'Wind_Inv_Cost',
       'Bioenergy_CCS_Inv_Cost', 'Other_Tech_Cost', 'Forcing', 'Land_Sinks',
       'Clim_Sens'],
    'all_inc_enc': ['Temp_Limit', 'Delay', 'GDP', 'Pop',
       'Other_ESD_Drivers', 'SDR', 'Elast_ESD_Driver', 'Elast_ESD_Price',
       'CO2_Storage_Poten', 'Wind_Poten', 'Solar_Poten', 'Biomass_Poten',
       'Oil_Gas_Poten', 'Solar_PV_Inv_Cost', 'Wind_Inv_Cost',
       'Bioenergy_CCS_Inv_Cost', 'Other_Tech_Cost', 'Forcing', 'Land_Sinks',
       'Clim_Sens']
}

scenarios = ['BASE_SSP2', '1p5c_OS_SSP2', '2C_SSP2', '2C_SSP2_DA30', 'ALL']

targets = ['GSupply', 'CO2eq', 'GConsumption', 'GCost'] # cannot do marginal CO2 as it's all-NaNs at 2050


In [9]:
def run_everything(Xy, cv=5):

    results = [] 

    for target in targets:
        for scenario in scenarios:
            if scenario == 'ALL':
                Xy_sub = Xy
            else:
                Xy_sub = Xy[Xy['Scenario'] == scenario]
            for xvar_subset_code in xvar_subsets:

                # for the 'ALL' scenario, run only if we have _inc_enc
                if scenario == 'ALL' and (not "_inc_enc" in xvar_subset_code): continue
                # for the non-ALL scenarios, run only if we DO NOT have _inc_enc
                if scenario != 'ALL' and ("_inc_enc" in xvar_subset_code): continue

                xvar_subset = xvar_subsets[xvar_subset_code]
                X = Xy_sub[xvar_subset]
                y = Xy_sub[target]
                print(f'Scenario {scenario}, xvar code {xvar_subset_code} xvars {xvar_subset}, target {target}')
                print("")


                models = [
                    ElasticNet,
                    RandomForestRegressor,
                    lambda: PySRRegressor(unary_operators=["sqrt"])

                    # would be nice to have several more eg:
                    # FFXRegressor,
                    # PyOperon,
                    # my custom SR implementation
                    # NN
                ]


                models = [m() for m in models]

                for m in models:
                    n = m.__class__.__name__
                    print(n)
                    scores = cross_validate(m, X, y, cv=cv,
                                            return_train_score=True)
                    # print(scores)
                    
                    
                    for fold, (tr, ts) in enumerate(zip(scores['train_score'], scores['test_score'])):
                        row = [target, scenario, xvar_subset_code, n, fold, tr, ts]

                        results.append(row)
    return results




In [ ]:
cv = 5
#results = run_everything(Xy, cv)
results_cols = ['Target', 'Scenario', 'xvar_subset_code', 'Regression Model', 'Fold', 'Train Score', 'Test Score']
results_df = pd.DataFrame(results, columns=results_cols)
results_df['Scenario / Variables'] = results_df['Scenario'] + ' / ' + results_df['xvar_subset_code']
#results_df.to_csv(f'../outputs/results_cv5_EN_RF_2xGP_clim_sens_filter_sqrt.csv')

Check that the results are the shape we expect:

In [11]:
results_df.head()

,Target,Scenario,xvar_subset_code,Regression Model,Fold,Train Score,Test Score,Scenario / Variables
0,GSupply,BASE_SSP2,small,ElasticNet,0,0.0,-0.015276,BASE_SSP2 / small
1,GSupply,BASE_SSP2,small,ElasticNet,1,0.0,-0.000236,BASE_SSP2 / small
2,GSupply,BASE_SSP2,small,ElasticNet,2,0.0,-0.003057,BASE_SSP2 / small
3,GSupply,BASE_SSP2,small,ElasticNet,3,0.0,-0.000131,BASE_SSP2 / small
4,GSupply,BASE_SSP2,small,ElasticNet,4,0.0,-0.000925,BASE_SSP2 / small


In [12]:
results_df.shape

(600, 8)

In [13]:
5 * 4 * 2 * 4 * 5 # 5 scenarios (incl ALL), 4 targets, 2 xvar subsets (variant in the case of ALL), 4 regression methods, 5 cv folds

800

# Statistics

In [14]:
from scipy.stats import ttest_ind

In [15]:
models = ['ElasticNet', 'RandomForestRegressor', 'PySRRegressor']
for model in models:
    sub = results_df[(results_df['Regression Model'] == model) & (results_df['Scenario'] != 'ALL')]['Test Score']
    print(f'{model} mean {sub.mean():.2f} std {sub.std():.1f}')
    for model2 in models:
        if model2 != model:
            sub2 = results_df[(results_df['Regression Model'] == model2) & (results_df['Scenario'] != 'ALL')]['Test Score']
            print(f'{model} {model2} {ttest_ind(sub, sub2)}')

ElasticNet mean 0.34 std 0.3
ElasticNet RandomForestRegressor TtestResult(statistic=-12.587304811321358, pvalue=9.318464867619149e-30, df=318.0)
ElasticNet PySRRegressor TtestResult(statistic=-10.934171065241669, pvalue=7.764711149562302e-24, df=318.0)
RandomForestRegressor mean 0.76 std 0.3
RandomForestRegressor ElasticNet TtestResult(statistic=12.587304811321358, pvalue=9.318464867619149e-30, df=318.0)
RandomForestRegressor PySRRegressor TtestResult(statistic=0.9210536332860737, pvalue=0.35772063132570475, df=318.0)
PySRRegressor mean 0.73 std 0.4
PySRRegressor ElasticNet TtestResult(statistic=10.934171065241669, pvalue=7.764711149562302e-24, df=318.0)
PySRRegressor RandomForestRegressor TtestResult(statistic=-0.9210536332860737, pvalue=0.35772063132570475, df=318.0)


In [16]:
for model in models:
    sub1 = results_df[(results_df['Regression Model'] == model) & (results_df['xvar_subset_code'] == 'small')]['Test Score']
    sub2 = results_df[(results_df['Regression Model'] == model) & (results_df['xvar_subset_code'] == 'all')]['Test Score']
    print(f'{model} small {sub1.mean():.2f} v all {sub2.mean():.2f}: {ttest_ind(sub1, sub2)}')

ElasticNet small 0.34 v all 0.34: TtestResult(statistic=-0.05775989959419327, pvalue=0.954012863737026, df=158.0)
RandomForestRegressor small 0.58 v all 0.95: TtestResult(statistic=-8.540990239490862, pvalue=1.0536351514575162e-14, df=158.0)
PySRRegressor small 0.52 v all 0.94: TtestResult(statistic=-9.120240976911184, pvalue=3.2899833152730717e-16, df=158.0)


In [17]:
# plot the individual scenario regression results
for target in targets:

    # we exclude GPAFSRegressor for now
    tmp = results_df[(results_df['Target'] == target) & (results_df['Regression Model'] != 'GPAFSRegressor') 
                     & (results_df['Scenario'] != 'ALL')]

    sns.stripplot(tmp, x='Scenario / Variables', y='Train Score', hue='Regression Model');
    plt.xticks(rotation=45, ha='right')
    plt.ylabel(f'Train $R^2$ on {target}')
    plt.tight_layout()
    plt.savefig(f'../outputs/results_boxplot_cv5_{target}_train_clim_sens_filter_sqrt.pdf')
    plt.close()

    sns.stripplot(tmp, x='Scenario / Variables', y='Test Score', hue='Regression Model');
    plt.xticks(rotation=45, ha='right')
    plt.ylabel(f'Test $R^2$ on {target}')
    plt.tight_layout()
    plt.savefig(f'../outputs/results_boxplot_cv5_{target}_test_clim_sens_filter_sqrt.pdf')
    plt.close()


In [18]:
# plot the *unified* scenario regression results
for target in targets:

    # we exclude GPAFSRegressor for now
    tmp = results_df[(results_df['Target'] == target) & (results_df['Regression Model'] != 'GPAFSRegressor')
                     & (results_df['Scenario'] == 'ALL')]

    sns.stripplot(tmp, x='Scenario / Variables', y='Train Score', hue='Regression Model');
    plt.xticks(rotation=45, ha='right')
    plt.ylabel(f'Train $R^2$ on {target}')
    plt.tight_layout()
    plt.savefig(f'../outputs/results_boxplot_cv5_{target}_unified_scenarios_train_clim_sens_filter_sqrt.pdf')
    plt.close()

    sns.stripplot(tmp, x='Scenario / Variables', y='Test Score', hue='Regression Model');
    plt.xticks(rotation=45, ha='right')
    plt.ylabel(f'Test $R^2$ on {target}')
    plt.tight_layout()
    plt.savefig(f'../outputs/results_boxplot_cv5_{target}_unified_scenarios_test_clim_sens_filter_sqrt.pdf')
    plt.close()


# Interpretation of models

## Variable importance and equation interpretation

In this section, we'll use a single train-test split and do a single run (per scenerio) for each of EN, RF, and SR
as our goal will not be a comparison of performance but an investigation of variable importance.

We include each scenario and all variables, not including the scenario-encoding variables.

RF gives variable importance directly.

EN does not, but we use abs(coefficient_i) * std(x_i)

SR we'll just store the equations together with their loss and complexity values.

In [ ]:
scenarios = ['BASE_SSP2', '1p5c_OS_SSP2', '2C_SSP2', '2C_SSP2_DA30']

import sympy_latex

for scenario in scenarios:
    Xy_tmp = Xy[Xy['Scenario'] == scenario]
    X = Xy_tmp[xvar_subsets['all']]
    for target in targets:
        y = Xy_tmp[target]
        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=1)

        for m in [#RandomForestRegressor(), 
                  #ElasticNet(), 
                  PySRRegressor(unary_operators=["sqrt"]), 
                  #GPAFSRegressor(20000, 4000, 0.5, 30, 5, "symbolic_regression.bnf", 0.3)]:
        ]:

            #m.fit(X_train, y_train)
            #m.score(X_test, y_test)

            if m.__class__.__name__ == 'RandomForestRegressor':
                # importance is given directly by the method
                imps = list(zip(m.feature_names_in_, m.feature_importances_))
                imps.sort(key=lambda x: x[1], reverse=True)
                imps = pd.DataFrame(imps, columns=['Feature', 'Importance'])
                # print, write importances to csv and to tex
                print(imps)
                imps.to_csv(f'../outputs/importances_rf_{scenario}_{target}_clim_sens_filter_sqrt.csv')
                imps['Feature'] = imps['Feature'].str.replace('_', ' ')
                imps.to_latex(f'../outputs/importances_rf_{scenario}_{target}_clim_sens_filter_sqrt.tex', index=False)     
            elif m.__class__.__name__ == 'ElasticNet':
                # importance is abs(coef) * std deviation of the var
                X_std = X_train.std(axis=0)
                imps = np.abs(m.coef_) * X_std
                imps = list(zip(m.feature_names_in_, imps))
                imps.sort(key=lambda x: x[1], reverse=True)
                imps = pd.DataFrame(imps, columns=['Feature', 'Importance'])
                # print, write importances to csv and to tex
                print(imps)
                imps.to_csv(f'../outputs/importances_en_{scenario}_{target}_clim_sens_filter_sqrt.csv')
                imps['Feature'] = imps['Feature'].str.replace('_', ' ')
                imps.to_latex(f'../outputs/importances_en_{scenario}_{target}_clim_sens_filter_sqrt.tex', index=False)
            elif m.__class__.__name__ == 'PySRRegressor' or m.__class__.__name__ == 'GPAFSRegressor':

                if m.__class__.__name__ == 'PySRRegressor': method = 'sr'
                if m.__class__.__name__ == 'GPAFSRegressor': method = 'magpie'

                # print, write original equations to csv
                #print(m.equations_)
                #m.equations_.to_csv(f'../outputs/equations_{method}_{scenario}_{target}_clim_sens_filter_sqrt.csv')
                # eqns = m.equations_

                eqns = pd.read_csv(f'../outputs/equations_{method}_{scenario}_{target}_clim_sens_filter_sqrt.csv')

                # write to latex table, but process first, and only useful columns
                eqns = sympy_latex.process_equations_latex(eqns)
                eqns = eqns[['vars', 'consts', 'complexity', 'loss', 'equation']]
                #eqns['equation'] = eqns['equation'].str.replace('_', ' ')
                eqns.to_latex(f'../outputs/equations_{method}_{scenario}_{target}_3cols_clim_sens_filter_sqrt.tex', index=False)


Oil_Gas_Poten
{Oil_Gas_Poten}
\mathrm{OilGasPoten} False
{\mathrm{OilGasPoten}}
\mathrm{OilGasPoten} 1 0
-----
sqrt(CO2_Storage_Poten)
{1/2, CO2_Storage_Poten}
1/2 True
\mathrm{CO2StoragePoten} False
{1/2, \mathrm{CO2StoragePoten}}
sqrt(\mathrm{CO2StoragePoten}) 1 1
-----
32.90138*Elast_ESD_Driver
{32.9013800000000, Elast_ESD_Driver}
32.90 True
\mathrm{ElastESDDriver} False
{32.90, \mathrm{ElastESDDriver}}
32.9*\mathrm{ElastESDDriver} 1 1
-----
38.37452*Elast_ESD_Driver - 5.496432
{Elast_ESD_Driver, -5.49643200000000, 38.3745200000000}
\mathrm{ElastESDDriver} False
-5.50 True
38.37 True
{\mathrm{ElastESDDriver}, -5.50, 38.37}
38.37*\mathrm{ElastESDDriver} - 5.5 1 2
-----
31.831852*Elast_ESD_Driver*sqrt(GDP)
{Elast_ESD_Driver, 1/2, GDP, 31.8318520000000}
\mathrm{GDP} False
1/2 True
\mathrm{ElastESDDriver} False
31.83 True
{\mathrm{GDP}, 1/2, \mathrm{ElastESDDriver}, 31.83}
31.83*\mathrm{ElastESDDriver}*sqrt(\mathrm{GDP}) 2 1
-----
37.25576*Elast_ESD_Driver*sqrt(GDP) - 5.718626
{Elast_ES